In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from PIL import Image
from tensorflow import keras
from tensorflow.keras import datasets,layers,optimizers,Sequential,metrics

In [2]:
def save_img(imgs,names):
  img_new = Image.new('L',(280,280))
  index = 0
  for i in range(0,280,80):
    for j in range(0,280,80):
      img = imgs[index]
      img = Image.fromarray(img,mode='L')
      img_new.paste(img,(i,j))
      index+=1
  img_new.save(names)

In [3]:
def feature_scale(x):
  x = tf.cast(x,dtype=tf.float32)/255.
#  y = tf.cast(y,dtype=tf.int32)
  return x

In [4]:
#Dim reduct nums
dim_reduce = 10
batch_num = 128
lr = 1e-3

In [5]:
(x,y),(x_test,y_test) = datasets.fashion_mnist.load_data()
data = tf.data.Dataset.from_tensor_slices(x)
data = data.map(feature_scale).shuffle(10000).batch(128)

data_test = tf.data.Dataset.from_tensor_slices(x_test)
data_test = data_test.map(feature_scale).batch(128)

In [6]:
class AE(keras.Model):
  def __init__(self):
    super(AE,self).__init__()
    #encoder
    self.model_encoder = Sequential([layers.Dense(256,activation=tf.nn.relu),
                             layers.Dense(128,activation=tf.nn.relu),
                             layers.Dense(dim_reduce,activation=tf.nn.relu) ])
    #decoder
    self.model_decoder = Sequential([layers.Dense(128,activation=tf.nn.relu),
                             layers.Dense(256,activation=tf.nn.relu),
                             layers.Dense(784,activation=tf.nn.relu)])

  def call(self, inputs, training=None):
    x = self.model_encoder(inputs)
    x = self.model_decoder(x)
    return x

In [7]:
model = AE()
model.build(input_shape=(None,784))
model.summary()

Model: "ae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 10)                235146    
_________________________________________________________________
sequential_1 (Sequential)    (None, 784)               235920    
Total params: 471,066
Trainable params: 471,066
Non-trainable params: 0
_________________________________________________________________


In [11]:
optimizer = optimizers.Adam(lr=lr)
for i in range(20):
  for step,x in enumerate(data):
    x = tf.reshape(x,[-1,784])
    with tf.GradientTape() as tape:
      logits = model(x)
      loss = tf.losses.binary_crossentropy(x,logits,from_logits=True)
      loss = tf.reduce_mean(loss)
    grads = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    
    if step %100==0:
      print(i,step,'loss:',float(loss))
      
  x = next(iter(data_test))
  val_x = tf.reshape(x,[-1,784])
  logits = model(val_x)
  x_hat = tf.sigmoid(logits)
  x_hat = tf.reshape(x_hat,[-1,28,28])
  x_hat = x_hat.numpy()*255
  x_hat = x_hat.astype(np.uint8)
  if i==19:
    save_img(x_hat,'./img_results/AE_img_%d.png'%i)

c:\Users\TMU\anaconda3\envs\tf2.6\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


0 0 loss: 0.6615171432495117
0 100 loss: 0.66971755027771
0 200 loss: 0.6667734384536743
0 300 loss: 0.6653379797935486
0 400 loss: 0.6657999753952026
1 0 loss: 0.6649190783500671
1 100 loss: 0.6637564301490784
1 200 loss: 0.6634138226509094
1 300 loss: 0.6610596179962158
1 400 loss: 0.6598784923553467
2 0 loss: 0.6652801632881165
2 100 loss: 0.6691753268241882
2 200 loss: 0.6642612218856812
2 300 loss: 0.6632861495018005
2 400 loss: 0.6679427623748779
3 0 loss: 0.6665161848068237
3 100 loss: 0.6635338068008423
3 200 loss: 0.6644402742385864
3 300 loss: 0.6630020141601562
3 400 loss: 0.6674457788467407
4 0 loss: 0.6629040241241455
4 100 loss: 0.6630797386169434
4 200 loss: 0.6669543981552124
4 300 loss: 0.6623529195785522
4 400 loss: 0.6647963523864746
5 0 loss: 0.6648557186126709
5 100 loss: 0.6655908226966858
5 200 loss: 0.6652087569236755
5 300 loss: 0.6637266874313354
5 400 loss: 0.6631309986114502
6 0 loss: 0.6608726978302002
6 100 loss: 0.6618208885192871
6 200 loss: 0.6649864315

In [10]:
x = next(iter(data_test))
val_x = tf.reshape(x,[-1,784])
logits = model(val_x)
x_pred = tf.sigmoid(logits)
x_pred = tf.reshape(x_pred,[-1,28,28])
x_pred = tf.concat([x,x_pred],axis=0)
x_pred = x_pred.numpy()*255
x_pred = x_pred.astype(np.uint8)
save_img(x_pred,'./img_results/AE_img_total.png')